In [1]:
from pettingzoo.mpe import simple_tag_v2
from gymnasium.utils.save_video import save_video

import time

In [2]:
env = simple_tag_v2.env(num_good=1, 
                        num_adversaries=3, 
                        num_obstacles=2, 
                        max_cycles=50, 
                        continuous_actions=True, 
                        render_mode="human"
)

In [11]:
env.reset()
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    action = None if termination or truncation else env.action_space(agent).sample()  # this is where you would insert your policy
    
    env.step(action)
    time.sleep(0.05)

env.render()
env.close()

Problems to solve: 
1. save videos [reference](https://github.com/fireofearth/cpsc533v_project/blob/c897af30921e13ac81ce01cc2a8066e1d46ecc31/colin/evaluate_tag_single_adversary.py)
3. try DQN
4. how to implement multiple network and train on that.

4. does the agent move fixed steps by each iteration in discrete mode
5. How to fix the landmark in fixed position, or is it just the rendering make it seem not staying where it is 


## Action, observation and State

In [5]:
s = env.reset()

# action space with four directions and stay

i = 0
# sample for different agents
for agent in env.agent_iter():
    a = env.action_space(agent).sample()
    print(agent, ':')
    # [no_action, move_left, move_right, move_down, move_up]
    print(env.action_space(agent))
    print(agent == 'adversary_0')
    env.step(a)
    print('sample action:', a)
    # print(env.observation_spaces)
    
    i = i + 1
    if i == 4:
        break

env.close()

adversary_0 :
Box(0.0, 1.0, (5,), float32)
True
sample action: [0.92901367 0.37396044 0.6046699  0.68962854 0.8919782 ]
adversary_1 :
Box(0.0, 1.0, (5,), float32)
False
sample action: [0.09669605 0.5639015  0.89365935 0.8282331  0.08617229]
adversary_2 :
Box(0.0, 1.0, (5,), float32)
False
sample action: [0.13027631 0.01165134 0.27997    0.39355087 0.16513759]
agent_0 :
Box(0.0, 1.0, (5,), float32)
False
sample action: [5.1456806e-04 9.9431151e-01 3.1835628e-01 4.3276712e-01 5.6209779e-01]


In [18]:
# state, should be 62?
env.state()

array([ 0.15303998,  0.03050993, -0.97915745, -0.28953764,  1.5197377 ,
        1.0614314 ,  1.2278101 ,  0.28194538,  0.6069736 ,  0.8174495 ,
        0.44835332,  0.46482474,  0.5754395 ,  1.1692449 ,  0.01615815,
       -0.00480299, -0.09565483,  0.11386324, -0.3721839 ,  0.5279119 ,
        0.9127642 ,  0.24398187,  0.6208366 , -0.53550416, -0.6069736 ,
       -0.8174495 , -0.15862025, -0.3526248 , -0.03153404,  0.35179535,
        0.01615815, -0.00480299, -0.14904232, -0.03848771, -0.53080416,
        0.17528708,  1.0713844 ,  0.5966067 ,  0.77945685, -0.18287934,
       -0.44835332, -0.46482474,  0.15862025,  0.3526248 ,  0.1270862 ,
        0.70442015,  0.01615815, -0.00480299,  0.01615815, -0.00480299,
       -0.40371794,  0.8797072 ,  0.94429827, -0.10781348,  0.65237063,
       -0.8872995 , -0.5754395 , -1.1692449 ,  0.03153404, -0.35179535,
       -0.1270862 , -0.70442015], dtype=float32)

In [3]:
s = env.reset()

i = 0
for agent in env.agent_iter():

    print(agent, ':')
    env.step(env.action_space(agent).sample())
    # observation space: shape(14),(16)
    # Agent and adversary observations: [self_vel, self_pos, landmark_rel_positions, other_agent_rel_positions, other_agent_velocities]
    # observation function in the simple_tag.py: other_agent_rel_positions contains all agents except oneself, the order is adversary to agent, 
    # other_agent_velocities only for not adversary
    print(env.observe(agent))
    
    i = i + 1
    if i == 8:
        break

env.close()

adversary_0 :
[ 0.          0.         -0.17424552 -0.08583507  0.56775844  0.3170014
  0.50590116 -0.35335577  1.0348797   0.9228235   0.781294   -0.19058752
  0.31464535  0.30438682  0.          0.        ]
adversary_1 :
[ 0.          0.          0.86063415  0.83698845 -0.46712124 -0.6058221
 -0.52897847 -1.2761793  -1.0348797  -0.9228235  -0.2535857  -1.1134111
 -0.72023433 -0.6184367   0.          0.        ]
adversary_2 :
[ 0.          0.          0.60704845 -0.2764226  -0.21353555  0.5075889
 -0.2753928  -0.16276824 -0.781294    0.19058752  0.2535857   1.1134111
 -0.46664864  0.49497434  0.          0.        ]
agent_0 :
[-0.03951665 -0.12613297  0.13644816  0.20593844  0.25706476  0.02522789
  0.19520749 -0.64512926 -0.32054192 -0.2870737   0.7219424   0.6249892
  0.49084908 -0.4678908 ]
adversary_0 :
[-0.09848244  0.04699823 -0.18409376 -0.08113524  0.5776067   0.31230158
  0.51574945 -0.3580556   1.0424844   0.9120629   0.811391   -0.1808171
  0.32054192  0.2870737  -0.0395166

## Policy network